## Observations and Insights 

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
#finalMouseData = study_results.set_index('Mouse ID').join(mouse_metadata.set_index('Mouse ID'), lsuffix='_prime', rsuffix='_secondary')
finalMouseData = pd.merge(study_results, mouse_metadata, on='Mouse ID', how='outer')

pd.set_option("display.max_rows", None, "display.max_columns", None)
# Display the data table for preview
print(finalMouseData)

     Mouse ID  Timepoint  Tumor Volume (mm3)  Metastatic Sites Drug Regimen  \
0        b128          0           45.000000                 0    Capomulin   
1        b128          5           45.651331                 0    Capomulin   
2        b128         10           43.270852                 0    Capomulin   
3        b128         15           43.784893                 0    Capomulin   
4        b128         20           42.731552                 0    Capomulin   
5        b128         25           43.262145                 1    Capomulin   
6        b128         30           40.605335                 1    Capomulin   
7        b128         35           37.967644                 1    Capomulin   
8        b128         40           38.379726                 2    Capomulin   
9        b128         45           38.982878                 2    Capomulin   
10       f932          0           45.000000                 0     Ketapril   
11       g107          0           45.000000        

In [2]:
# Checking the number of mice.
finaDataMouseCount = len(finalMouseData['Mouse ID'].unique())

finaDataMouseCount

249

In [7]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
mouseIds = finalMouseData[['Mouse ID', 'Timepoint']]
#print(finalMouseData[mouseIds.isin(mouseIds[mouseIds.duplicated(keep=False)])])
duplicateIds = finalMouseData[mouseIds.isin(mouseIds[mouseIds.duplicated(keep=False)])]
#duplicateMice = finalMouseData.duplicated(['Mouse ID', 'Timepoint'])

#print(duplicateIds[['Mouse ID', 'Timepoint']].dropna())
duplicateIds = duplicateIds[['Mouse ID', 'Timepoint']].dropna()
duplicateJustIds = duplicateIds['Mouse ID'].unique()

#duplicateFreeData = finalMouseData.drop_duplicates(keep=False, subset=['Mouse ID', 'Timepoint'])
#print(duplicateFreeData)
#print(duplicateJustIds)
#duplicatesDroppedMiceData = finalMouseData[~finalMouseData['Mouse ID'].isin(duplicateJustIds)]
#duplicateMiceData = finalMouseData[finalMouseData['Mouse ID'].isin(duplicateJustIds)]
#print(duplicatesDroppedMiceData)

     Mouse ID  Timepoint  Tumor Volume (mm3)  Metastatic Sites Drug Regimen  \
0        b128          0           45.000000                 0    Capomulin   
1        b128          5           45.651331                 0    Capomulin   
2        b128         10           43.270852                 0    Capomulin   
3        b128         15           43.784893                 0    Capomulin   
4        b128         20           42.731552                 0    Capomulin   
5        b128         25           43.262145                 1    Capomulin   
6        b128         30           40.605335                 1    Capomulin   
7        b128         35           37.967644                 1    Capomulin   
8        b128         40           38.379726                 2    Capomulin   
9        b128         45           38.982878                 2    Capomulin   
10       f932          0           45.000000                 0     Ketapril   
11       g107          0           45.000000        

In [8]:
# Optional: Get all the data for the duplicate mouse ID. 
duplicateMiceData = finalMouseData[finalMouseData['Mouse ID'].isin(duplicateJustIds)]


In [9]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
duplicatesDroppedMiceData = finalMouseData[~finalMouseData['Mouse ID'].isin(duplicateJustIds)]

In [11]:
# Checking the number of mice in the clean DataFrame.
cleanMouseCount = len(duplicatesDroppedMiceData['Mouse ID'].unique())

print(cleanMouseCount)

248


## Summary Statistics

In [7]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
# Assemble the resulting series into a single summary dataframe.



In [8]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Using the aggregation method, produce the same summary statistics in a single line


## Bar and Pie Charts

In [9]:
# Generate a bar plot showing the total number of unique mice tested on each drug regimen using pandas.



In [10]:
# Generate a bar plot showing the total number of unique mice tested on each drug regimen using pyplot.



In [11]:
# Generate a pie plot showing the distribution of female versus male mice using pandas



In [12]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot



## Quartiles, Outliers and Boxplots

In [13]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint


In [14]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [15]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest


## Line and Scatter Plots

In [16]:
# Generate a line plot of tumor volume vs. time point for a mouse treated with Capomulin


In [17]:
# Generate a scatter plot of average tumor volume vs. mouse weight for the Capomulin regimen


## Correlation and Regression

In [18]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
